In [2]:
import pickle
import pandas as pd

with open('knn_model.pkl', 'rb') as file:
    knn_model = pickle.load(file)

In [21]:
#newdataframe
new_df = pd.read_csv('predict_attack.csv')

# Extract the feature columns from the new DataFrame and convert to numpy array
X_new = new_df[['Src_Port',
 'Dst_Port',
 'Protocol',
 'Flow_Duration',
 'Tot_Fwd_Pkts',
 'Tot_Bwd_Pkts',
 'TotLen_Fwd_Pkts',
 'TotLen_Bwd_Pkts',
 'Fwd_Pkt_Len_Max',
 'Fwd_Pkt_Len_Min',
 'Fwd_Pkt_Len_Mean',
 'Fwd_Pkt_Len_Std',
 'Bwd_Pkt_Len_Max',
 'Bwd_Pkt_Len_Min',
 'Bwd_Pkt_Len_Mean',
 'Bwd_Pkt_Len_Std',
 'Flow_IAT_Mean',
 'Flow_IAT_Std',
 'Flow_IAT_Max',
 'Flow_IAT_Min',
 'Fwd_IAT_Tot',
 'Fwd_IAT_Mean',
 'Fwd_IAT_Std',
 'Fwd_IAT_Max',
 'Fwd_IAT_Min',
 'Bwd_IAT_Tot',
 'Bwd_IAT_Mean',
 'Bwd_IAT_Std',
 'Bwd_IAT_Max',
 'Bwd_IAT_Min',
 'Fwd_PSH_Flags',
 'Bwd_PSH_Flags',
 'Fwd_URG_Flags',
 'Bwd_URG_Flags',
 'Fwd_Header_Len',
 'Bwd_Header_Len',
 'Fwd_Pkts/s',
 'Bwd_Pkts/s',
 'Pkt_Len_Min',
 'Pkt_Len_Max',
 'Pkt_Len_Mean',
 'Pkt_Len_Std',
 'Pkt_Len_Var',
 'FIN_Flag_Cnt',
 'SYN_Flag_Cnt',
 'RST_Flag_Cnt',
 'PSH_Flag_Cnt',
 'ACK_Flag_Cnt',
 'URG_Flag_Cnt',
 'CWE_Flag_Count',
 'ECE_Flag_Cnt',
 'Down/Up_Ratio',
 'Pkt_Size_Avg',
 'Fwd_Seg_Size_Avg',
 'Bwd_Seg_Size_Avg',
 'Fwd_Byts/b_Avg',
 'Fwd_Pkts/b_Avg',
 'Fwd_Blk_Rate_Avg',
 'Bwd_Byts/b_Avg',
 'Bwd_Pkts/b_Avg',
 'Bwd_Blk_Rate_Avg',
 'Subflow_Fwd_Pkts',
 'Subflow_Fwd_Byts',
 'Subflow_Bwd_Pkts',
 'Subflow_Bwd_Byts',
 'Init_Fwd_Win_Byts',
 'Init_Bwd_Win_Byts',
 'Fwd_Act_Data_Pkts',
 'Fwd_Seg_Size_Min',
 'Active_Mean',
 'Active_Std',
 'Active_Max',
 'Active_Min',
 'Idle_Mean',
 'Idle_Std',
 'Idle_Max',
 'Idle_Min',
 'S_octet1',
 'S_octet2',
 'S_octet3',
 'S_octet4',
 'D_octet1',
 'D_octet2',
 'D_octet3',
 'D_octet4']].values

new_predictions = knn_model.predict(X_new)

# Add the predicted output features to the new DataFrame
new_df['output_Label'] = new_predictions[:, 0]
new_df['output_Cat'] = new_predictions[:, 1]

# Write the new DataFrame with predicted output features to a new CSV file
new_df.to_csv('results.csv', index=False)

In [3]:
import smtplib

# read the csv file
new_df3 = pd.read_csv('results.csv')

# count the number of anomalies
num_anomalies = new_df3['output_Label'].str.contains('Anomaly').sum()

# calculate the percentage of anomalies
anomaly_pct = num_anomalies / len(new_df3)

sender = "xosepo2109@meidecn.com"
receiver = "alerttheuser@gmail.com"

message = f"""\
Subject: Anomaly Detected!
To: {receiver}
From: {sender}

IDS has detected {num_anomalies} Anomalies, Urgent visit the dashboard at http://10.122.168.123:5000








2023 The IDS, All Rights Reserved.
"""

# if the percentage is greater than 1%, show the anomalies
if anomaly_pct > 0.01:
    # read the csv file
    new_df2 = pd.read_csv('results.csv')

    # filter the rows where output_Label contains 'anomaly'
    filtered_df = new_df2[new_df2['output_Label'].str.contains('Anomaly')]

    # select only the required columns
    filtered_df = filtered_df[['Id','Timestamp','Dst_IP', 'Src_IP', 'output_Label', 'output_Cat']]

    # Write the new DataFrame
    filtered_df.to_csv('results.csv', index=False)

    #playsound.playsound('alert.wav', True)
    #time.sleep(5)


    
    with smtplib.SMTP("sandbox.smtp.mailtrap.io", 2525) as server:
        server.login("", "")
        server.sendmail(sender, receiver, message)
    # print the filtered dataframe
    print(filtered_df)
else:

    # print the filtered dataframe
    print("No Anomalies Detected!")

    Id               Timestamp           Dst_IP           Src_IP output_Label   
0    0  25/07/2019 03:25:53 AM     192.168.0.16     192.168.0.13      Anomaly  \
1    1  26/05/2019 10:11:06 PM     192.168.0.13  222.160.179.132      Anomaly   
2    2  11/07/2019 01:24:48 AM     192.168.0.13     192.168.0.16      Anomaly   
3    3  04/09/2019 03:58:17 AM     192.168.0.13     192.168.0.16      Anomaly   
4    4  10/09/2019 01:41:18 AM  239.255.255.250      192.168.0.1      Anomaly   
5    5  10/09/2019 01:39:13 AM   101.79.244.148     192.168.0.24      Anomaly   
6    6  25/07/2019 03:21:01 AM    210.89.164.90     192.168.0.24      Anomaly   
7    7  11/07/2019 01:52:37 AM     58.225.75.83     192.168.0.24      Anomaly   
8    8  25/07/2019 03:21:13 AM    210.89.164.90     192.168.0.13      Anomaly   
9    9  26/05/2019 10:20:36 PM     192.168.0.13  111.149.163.151      Anomaly   
10  10  04/09/2019 03:58:32 AM     40.100.49.34     192.168.0.24      Anomaly   
11  11  25/07/2019 03:26:18 